## Preprocessing

In [1]:
# Using Imports from class that has all of the imports for all the models
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

# Metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, RocCurveDisplay

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Tensorflow
import tensorflow as tf
from tensorflow.keras.layers import LeakyReLU

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import and read the charity_data.csv.
charity_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
charity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
drop_columns = ["EIN", "NAME"]
charity_df.drop(columns=drop_columns, inplace=True)
charity_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
charity_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [6]:
# Determine the number of unique values in each column.
cols = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'IS_SUCCESSFUL']

for col in cols:
  print(col)
  print(charity_df[col].nunique())
  print(charity_df[col].value_counts())
  print()

APPLICATION_TYPE
17
APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

AFFILIATION
6
AFFILIATION
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: count, dtype: int64

CLASSIFICATION
71
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

USE_CASE
5
USE_CASE
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: count, dtype: int64

ORGANIZATION
4
ORGANIZATION
Trust           23515
Association     10255
Co-operative      486
Corporation  

In [7]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
# Choose a cutoff value and create a list of application types to be replaced.  Cutoff chosen was 525.
app_type_counts =charity_df["APPLICATION_TYPE"].value_counts()
app_type_counts.loc[app_type_counts < 525].index.tolist()

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [8]:
# use the variable name `application_types_to_replace`
cutoff = 275
application_types_to_replace = app_type_counts.loc[app_type_counts < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    charity_df['APPLICATION_TYPE'] = charity_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
charity_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [9]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = charity_df["CLASSIFICATION"].value_counts()
classification_counts.loc[classification_counts > 1]

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 275
classifications_to_replace = classification_counts.loc[classification_counts < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    charity_df['CLASSIFICATION'] = charity_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
charity_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
Other,1197
C7000,777
C1700,287


In [11]:
# Look at USE_CASE value counts to identify and replace with "Other"
use_case_counts = charity_df["USE_CASE"].value_counts()
use_case_counts.loc[use_case_counts > 1]

,count
USE_CASE,
Preservation,28095
ProductDev,5671
CommunityServ,384
Heathcare,146
Other,3


In [12]:
# use the variable name `use_case_to_replace`
cutoff = 375
use_case_to_replace = use_case_counts.loc[use_case_counts < cutoff].index.tolist()

# Replace in dataframe
for app in use_case_to_replace:
    charity_df['USE_CASE'] = charity_df['USE_CASE'].replace(app,"Other")

# Check to make sure replacement was successful
charity_df['USE_CASE'].value_counts()

,count
USE_CASE,
Preservation,28095
ProductDev,5671
CommunityServ,384
Other,149


In [13]:
# Look at ORGANIZATION value counts to identify and replace with "Other"
organization_counts = charity_df["ORGANIZATION"].value_counts()
organization_counts.loc[organization_counts > 1]

,count
ORGANIZATION,
Trust,23515
Association,10255
Co-operative,486
Corporation,43


In [14]:
# use the variable name `organization_to_replace`
cutoff = 10000
organization_to_replace = organization_counts.loc[organization_counts < cutoff].index.tolist()

# Replace in dataframe
for app in organization_to_replace:
    charity_df['ORGANIZATION'] = charity_df['ORGANIZATION'].replace(app,"Other")

# Check to make sure replacement was successful
charity_df['ORGANIZATION'].value_counts()

,count
ORGANIZATION,
Trust,23515
Association,10255
Other,529


In [15]:
# Determine the number of unique values in each column.
cols = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'IS_SUCCESSFUL']

for col in cols:
  print(col)
  print(charity_df[col].nunique())
  print(charity_df[col].value_counts())
  print()

APPLICATION_TYPE
9
APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

AFFILIATION
6
AFFILIATION
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: count, dtype: int64

CLASSIFICATION
8
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: count, dtype: int64

USE_CASE
4
USE_CASE
Preservation     28095
ProductDev        5671
CommunityServ      384
Other              149
Name: count, dtype: int64

ORGANIZATION
3
ORGANIZATION
Trust          23515
Association    10255
Other            529
Name: count, dtype: int64

STATUS
2
STATUS
1    34294
0        5
Name: count, dtype: int64

INCOME_AMT
9
INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374

In [16]:
# Convert categorical data to numeric with `pd.get_dummies`
charity_df2 = pd.get_dummies(charity_df, dtype=int)
charity_df2.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1700,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0


In [17]:
# Split our preprocessed data into our features and target arrays
X = charity_df2.drop(columns=["IS_SUCCESSFUL"])
y = charity_df2.IS_SUCCESSFUL

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y) # stratify=True maintains target class percentages
print(X_train.shape, X_test.shape)

(25724, 43) (8575, 43)


In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [19]:
# Create the Keras Sequential model.  I also tried with one layer, 2 layers and added even more layers without improvement in order to reach 75%.  At the bottom of the Notebook there are also other things I tried (TANH, LeakyRelu)
nn = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=len(X.columns)))

# Add 2nd Layer
nn.add(tf.keras.layers.Dense(units=4, activation="relu", input_dim=len(X.columns)))

# Add 3rd Layer
nn.add(tf.keras.layers.Dense(units=2, activation="relu", input_dim=len(X.columns)))

# Add the output layer that uses a probability activation function
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 8)                   │             352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │              10 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │               3 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 401 (1.57 KB)

 Trainable params: 401 (1.57 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
#Used Chatgpt to help with saving weights function
class SaveWeightsEveryNEpochs(tf.keras.callbacks.Callback):
    def __init__(self, save_freq, save_path):
        super(SaveWeightsEveryNEpochs, self).__init__()
        self.save_freq = save_freq
        self.save_path = save_path

    def on_epoch_end(self, epoch, logs=None):
        # Save weights every `save_freq` epochs
        if (epoch + 1) % self.save_freq == 0:
            # Save weights using the correct method
            self.model.save_weights(f'{self.save_path}_epoch_{epoch + 1}.weights.h5')
            print(f'\nEpoch {epoch + 1}: Saving weights to {self.save_path}_epoch_{epoch + 1}.weights.h5')

In [22]:
# Define the custom callback
save_weights_callback = SaveWeightsEveryNEpochs(save_freq=5, save_path='model_weights')

# Fit the model with the custom callback - I also used less EPOCHS (8, 10) and More EPOCHS (up to 50). I was unable to reah 75%.
history = nn.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[save_weights_callback]
)

Epoch 1/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.5281 - loss: 0.6922 - val_accuracy: 0.5287 - val_loss: 0.6915
Epoch 2/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5222 - loss: 0.6922 - val_accuracy: 0.5287 - val_loss: 0.6916
Epoch 3/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5304 - loss: 0.6914 - val_accuracy: 0.5287 - val_loss: 0.6916
Epoch 4/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5329 - loss: 0.6910 - val_accuracy: 0.5287 - val_loss: 0.6916
Epoch 5/10
631/644 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5295 - loss: 0.6915
Epoch 5: Saving weights to model_weights_epoch_5.weights.h5
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5296 - loss: 0.6915 - val_accuracy: 0.5287 - val_loss: 0.6916
Epoch 6/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5319 - loss: 0.6912 - val_accuracy: 0.5287 - val_loss: 0.6916
Epoch 7/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5362 - loss: 0.6906 - val_accu

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 3ms/step - accuracy: 0.4033 - loss: 0.7418
Loss: 0.7418286204338074, Accuracy: 0.4032652974128723


In [24]:
# Create the Keras Sequential model
nn2 = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn2.add(tf.keras.layers.Dense(units=12, activation="relu", input_dim=len(X.columns)))

# Add the output layer that uses a probability activation function
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 12)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              13 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 541 (2.11 KB)

 Trainable params: 541 (2.11 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
 #Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Define the custom callback
save_weights_callback = SaveWeightsEveryNEpochs(save_freq=5, save_path='model_weights')

# Fit the model with the custom callback - I also used less EPOCHS (8, 10) and More EPOCHS (up to 50). I was unable to reah 75%.
history = nn2.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[save_weights_callback]
)

Epoch 1/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4787 - loss: 16369.7207 - val_accuracy: 0.5287 - val_loss: 30538.2988
Epoch 2/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4975 - loss: 35532.3477 - val_accuracy: 0.5287 - val_loss: 25209.9863
Epoch 3/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4943 - loss: 74617.9062 - val_accuracy: 0.4713 - val_loss: 17311.1152
Epoch 4/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4919 - loss: 13931.5635 - val_accuracy: 0.5287 - val_loss: 41412.5469
Epoch 5/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5130 - loss: 36539.6328
Epoch 5: Saving weights to model_weights_epoch_5.weights.h5
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5130 - loss: 36539.8945 - val_accuracy: 0.5238 - val_loss: 205.7924
Epoch 6/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.4987 - loss: 10822.5840 - val_accuracy: 0.4713 - val_loss: 6868.8340
Epoch 7/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/s

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 3ms/step - accuracy: 0.7030 - loss: 2.8898
Loss: 2.889761447906494, Accuracy: 0.7029737830162048


In [28]:
# Create the Keras Sequential model
nn3 = tf.keras.models.Sequential()

# Add the first Dense layer with LeakyReLU activation
nn3.add(tf.keras.layers.Dense(units=8, input_dim=len(X.columns)))
nn3.add(tf.keras.layers.LeakyReLU(alpha=0.01))  # You can adjust the alpha value

# Add the output layer that uses a probability activation function
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn3.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 8)                   │             352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 361 (1.41 KB)

 Trainable params: 361 (1.41 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
#Tried LeakyRelu - accuracy was just slightly better, more overfit


nn.add(tf.keras.layers.Dense(units=20, input_dim=len(X.columns)))
nn.add(LeakyReLU(alpha=0.01))
nn.add(tf.keras.layers.Dense(units=16))
nn.add(LeakyReLU(alpha=0.01))
nn.add(tf.keras.layers.Dense(units=12))
nn.add(LeakyReLU(alpha=0.01))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [31]:
# Define the custom callback
save_weights_callback = SaveWeightsEveryNEpochs(save_freq=5, save_path='model_weights')

# Fit the model with the custom callback - I also used less EPOCHS (8, 10) and More EPOCHS (up to 50). I was unable to reah 75%.
history = nn3.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[save_weights_callback]
)

Epoch 1/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5336 - loss: 339225.5625 - val_accuracy: 0.5287 - val_loss: 6995.4805
Epoch 2/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4948 - loss: 4656.4604 - val_accuracy: 0.5287 - val_loss: 14051.8018
Epoch 3/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4994 - loss: 7862.8564 - val_accuracy: 0.5287 - val_loss: 20832.3340
Epoch 4/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5157 - loss: 37308.1055 - val_accuracy: 0.4713 - val_loss: 22159.0586
Epoch 5/10
637/644 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4741 - loss: 36986.5586
Epoch 5: Saving weights to model_weights_epoch_5.weights.h5
644/644 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.4742 - loss: 36791.9102 - val_accuracy: 0.4713 - val_loss: 6911.3672
Epoch 6/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4936 - loss: 14128.8203 - val_accuracy: 0.5553 - val_loss: 878.6851
Epoch 7/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/ste

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7039 - loss: 3.9524
Loss: 3.952415704727173, Accuracy: 0.7039067149162292


In [33]:
# Create the Keras Sequential model.  I also tried with one layer, 2 layers and added even more layers without improvement in order to reach 75%.  At the bottom of the Notebook there are also other things I tried (TANH, LeakyRelu)
nnbest = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nnbest.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=len(X.columns)))

# Add the output layer that uses a probability activation function
nnbest.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nnbest.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 8)                   │             352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 361 (1.41 KB)

 Trainable params: 361 (1.41 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
# Compile the model
nnbest.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Define the custom callback
save_weights_callback = SaveWeightsEveryNEpochs(save_freq=5, save_path='model_weights')

# Fit the model with the custom callback - I also used less EPOCHS (8, 10) and More EPOCHS (up to 50). I was unable to reah 75%.
history = nnbest.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[save_weights_callback]
)

Epoch 1/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4951 - loss: 12808.2461 - val_accuracy: 0.4713 - val_loss: 2624.6411
Epoch 2/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5077 - loss: 882.2761 - val_accuracy: 0.4713 - val_loss: 16966.6660
Epoch 3/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4722 - loss: 8568.8574 - val_accuracy: 0.5287 - val_loss: 22819.6973
Epoch 4/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5182 - loss: 14348.9717 - val_accuracy: 0.5287 - val_loss: 1713.3088
Epoch 5/10
615/644 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5369 - loss: 7625.4844
Epoch 5: Saving weights to model_weights_epoch_5.weights.h5
644/644 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5357 - loss: 8393.9473 - val_accuracy: 0.4713 - val_loss: 12569.5557
Epoch 6/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4795 - loss: 9634.4980 - val_accuracy: 0.4713 - val_loss: 1979.3804
Epoch 7/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - 

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nnbest.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7031 - loss: 1.5352
Loss: 1.5352212190628052, Accuracy: 0.7030903697013855


In [39]:
# Export our model to HDF5 file
nnbest.save("AlphabetSoupCharity.keras")

Some of the EXTRA Optimization Techniques I tried

In [ ]:
# Create the Keras Sequential model.  I also tried with one layer, 2 layers and added even more layers without improvement in order to reach 75%.  At the bottom of the Notebook there are also other things I tried (TANH, LeakyRelu)
nn = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=len(X.columns)))

# Add the output layer that uses a probability activation function
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn.summary()

In [ ]:
#Tried tanh, yes worse as I expected
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="tanh", input_dim=len(X.columns)))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=12, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


In [ ]:
#Used this However then Loss was worse with more overfitting
nn = tf.keras.models.Sequential()

# Add more neurons and layers
nn.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=len(X.columns)))
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn.summary()
